In [1]:
!pip install pandas numpy scikit-learn nltk spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
import pandas as pd
import numpy as np
import pickle
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import bigrams
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")
from sklearn.decomposition import NMF
from sklearn.metrics import pairwise_distances

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
df = pd.read_csv("/content/recipes_cleaned_with_ids.csv")

In [25]:
df1=pd.read_csv("/content/just_ingredients.csv")

In [5]:
def commatokenizer(text):
    return text.split(', ')

def get_nouns(text):
    tokens = RegexpTokenizer(r'\w+').tokenize(text)
    nouns = {'NN', 'NNS', 'NNP', 'NNPS', 'NOUN', 'PROPN', 'NE', 'NNE', 'NR'}
    nounlist = [token for token in tokens if nlp(token)[0].tag_ in nouns]
    return ', '.join(nounlist)

def mytokenizer(combinedlist):
    ingredlist = combinedlist[0].split(', ')
    nounlist = combinedlist[1].split(', ')
    bigramlist = []
    for ingred in ingredlist:
        bigrms = [bi for bi in bigrams(ingred.split())]
        for bi in bigrms:
            if (bi[0] in nounlist) or (bi[1] in nounlist):
                bigramlist.append(' '.join((bi[0], bi[1])))
    return ', '.join(bigramlist + nounlist)

In [6]:
def process_ingredients(row):
    nouns = get_nouns(row)
    combined = [row, nouns]
    return mytokenizer(combined)

In [7]:
df1['TokenizedIngredients'] = df1['IngredientsRemovedAdj'].apply(process_ingredients)

In [8]:
def user_tokenize(ingreds):
    nouns = get_nouns(ingreds)
    ingredscombined = [ingreds, nouns]
    ingredstokenized = mytokenizer(ingredscombined)
    return ingredstokenized

In [9]:
vectorizer = TfidfVectorizer(tokenizer=commatokenizer, stop_words='english', min_df=7, max_df=0.4)
docs = df1['TokenizedIngredients']
doc_word = vectorizer.fit_transform(docs)

print(doc_word.shape)

(1097, 249)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
nmf_model = NMF(20, random_state=10, max_iter=1000)
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

In [19]:
from sklearn.decomposition import NMF

# Experiment with different numbers of topics
n_topics = [15, 20, 25, 30]
best_nmf_model = None
best_score = float('inf')

for n in n_topics:
    nmf_model = NMF(n, random_state=10, max_iter=2000)
    doc_topic = nmf_model.fit_transform(doc_word)
    topic_word = nmf_model.components_
    score = nmf_model.reconstruction_err_

    if score < best_score:
        best_score = score
        best_nmf_model = nmf_model

nmf_model = best_nmf_model
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

def display_topics(model, feature_names, num_top_words, topic_names=None):
    for idx, topic in enumerate(model.components_):
        if not topic_names or not topic_names[idx]:
            print("\nTopic ", idx)
        else:
            print("\nTopic: '", topic_names[idx], "'")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model, vectorizer.get_feature_names_out(), 8)



Topic  0
brown, brown sugar, sugar, cinnamon, oat, butter, pecan, vanilla

Topic  1
pepper, black pepper, garlic, butter, cayenne pepper, cayenne, onion, parsley

Topic  2
stone, stone house, house, house seasoning, parsley, ranch, bread, chili powder

Topic  3
cheddar, cheddar cheese, cheese, bacon, mayonnaise, ham, green onion, milk

Topic  4
olive, olive oil, oil, zucchini, pork, shrimp, garlic, pesto

Topic  5
powder, baking powder, flour, sugar, butter, vanilla, vanilla extract, cinnamon

Topic  6
vanilla, vanilla extract, milk, sugar, cocoa, butter, espresso, condensed milk

Topic  7
chicken, stock, chicken stock, broth, soup, thyme, celery, mushroom

Topic  8
water, yeast, corn, ice, sugar, corn syrup, rice, vegetable

Topic  9
peanut, peanut butter, butter, oat, chocolate chip, honey, chip, vanilla ice

Topic  10
self, self rising, rising flour, flour, milk, butter, cornmeal, yeast

Topic  11
cream, cream cheese, cheese, sour cream, whipping cream, ice cream, ice, vanilla ice


ngiiiiiiiide

In [26]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from nltk import bigrams
from nltk.tokenize import RegexpTokenizer
import spacy

# Download NLTK stopwords if not already downloaded
import nltk
nltk.download('stopwords')

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function definitions for tokenization and processing
def commatokenizer(text):
    return text.split(', ')

def get_nouns(text):
    tokens = RegexpTokenizer(r'\w+').tokenize(text)
    nouns = {'NN', 'NNS', 'NNP', 'NNPS', 'NOUN', 'PROPN', 'NE', 'NNE', 'NR'}
    nounlist = [token for token in tokens if nlp(token)[0].tag_ in nouns]
    return ', '.join(nounlist)

def mytokenizer(combinedlist):
    ingredlist = combinedlist[0].split(', ')
    nounlist = combinedlist[1].split(', ')
    bigramlist = []
    for ingred in ingredlist:
        bigrms = [bi for bi in bigrams(ingred.split())]
        for bi in bigrms:
            if (bi[0] in nounlist) or (bi[1] in nounlist):
                bigramlist.append(' '.join((bi[0], bi[1])))
    return ', '.join(bigramlist + nounlist)

def process_ingredients(row):
    nouns = get_nouns(row)
    combined = [row, nouns]
    return mytokenizer(combined)

def display_topics(model, feature_names, num_top_words, topic_names=None):
    for idx, topic in enumerate(model.components_):
        if not topic_names or not topic_names[idx]:
            print("\nTopic ", idx)
        else:
            print("\nTopic: '", topic_names[idx], "'")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Read data from CSV files
df = pd.read_csv("/content/recipes_cleaned_with_ids.csv")
df1 = pd.read_csv("/content/just_ingredients.csv")

# Tokenize ingredients using process_ingredients function
df1['TokenizedIngredients'] = df1['IngredientsRemovedAdj'].apply(process_ingredients)

# Vectorize tokenized ingredients using TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=commatokenizer, stop_words='english', min_df=7, max_df=0.4)
docs = df1['TokenizedIngredients']
doc_word = vectorizer.fit_transform(docs)

print("Shape of doc_word matrix:", doc_word.shape)

# Experiment with different numbers of topics using NMF
n_topics = [15, 20, 25, 30]
best_nmf_model = None
best_score = float('inf')

for n in n_topics:
    nmf_model = NMF(n_components=n, random_state=10, max_iter=2000)
    doc_topic = nmf_model.fit_transform(doc_word)
    score = nmf_model.reconstruction_err_

    if score < best_score:
        best_score = score
        best_nmf_model = nmf_model

# Use the best NMF model found
nmf_model = best_nmf_model
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

# Display topics from the best NMF model
display_topics(nmf_model, vectorizer.get_feature_names_out(), 8)

# Define weights for key ingredients
key_ingredients_weights = {
    'beef': 10,
    'chicken': 10,
    'shrimp': 10,
    'crab': 10,
    'venison': 10,
    # Add more key ingredients and their specific weights as needed
}

# Further processing or analysis based on the results


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Shape of doc_word matrix: (1097, 249)

Topic  0
brown, brown sugar, sugar, cinnamon, oat, butter, pecan, vanilla

Topic  1
pepper, black pepper, garlic, butter, cayenne pepper, cayenne, onion, parsley

Topic  2
stone, stone house, house, house seasoning, parsley, ranch, bread, chili powder

Topic  3
cheddar, cheddar cheese, cheese, bacon, mayonnaise, ham, green onion, milk

Topic  4
olive, olive oil, oil, zucchini, pork, shrimp, garlic, pesto

Topic  5
powder, baking powder, flour, sugar, butter, vanilla, vanilla extract, cinnamon

Topic  6
vanilla, vanilla extract, milk, sugar, cocoa, butter, espresso, condensed milk

Topic  7
chicken, stock, chicken stock, broth, soup, thyme, celery, mushroom

Topic  8
water, yeast, corn, ice, sugar, corn syrup, rice, vegetable

Topic  9
peanut, peanut butter, butter, oat, chocolate chip, honey, chip, vanilla ice

Topic  10
self, self rising, rising flour, flour, milk, butter, cornmeal, yeast

Topic  11
cream, cream cheese, cheese, sour cream, whippi

In [20]:

key_ingredients_weights = {
    'beef': 10,
    'chicken': 10,
    'shrimp': 10,
    'crab': 10,
    'venison': 10,
    # Add more key ingredients and their specific weights
}


INI YG TOP 5


In [43]:
useringreds = "corn"
usertokens = user_tokenize(useringreds)
print('User Input: ', useringreds)
print('Tokens Generated: ', usertokens, '\n')

# Vectorize user input
user_vec = vectorizer.transform([usertokens])

# Adjust weights for key ingredients
feature_names = vectorizer.get_feature_names_out()
for ingredient, weight in key_ingredients_weights.items():
    if ingredient in usertokens:
        index = feature_names.tolist().index(ingredient)
        user_vec[0, index] *= weight

# Transform user vector into topic space
topic_vec = nmf_model.transform(user_vec)

# Compute similarity and get recommendations
indices = pairwise_distances(topic_vec, doc_topic, metric='cosine').argsort().ravel()

print("Top 5 Recipe Recommendations containing '{}' or its tokens:".format(useringreds))
found_recommendation = 0
for index in indices:
    if found_recommendation >= 5:
        break
    if any(token in df.iloc[index]['All Ingredients'].lower() for token in usertokens.split(', ')):
        print(df.iloc[index].Title.upper())
        print(df.iloc[index]["All Ingredients"])
        print()  # Print a blank line for readability
        found_recommendation += 1

if found_recommendation == 0:
    print("No recipes found containing '{}' or its tokens.".format(useringreds))


User Input:  corn
Tokens Generated:  corn 

Top 5 Recipe Recommendations containing 'corn' or its tokens:
TOFFEE PECAN PIE RECIPE
1 Pie Crust Recipe, 3 eggs, ½ cup sugar, 1 cup corn syrup, 1 teaspoon vanilla extract, 1/4 cup butter (melted), pinch kosher salt, 2 cups pecans, 1 cup toffee bits

BEST PECAN PIE RECIPE
½ cup granulated sugar, 3 large eggs, 1 cup light corn syrup or maple syrup, ½ cup butter (melted), 1 teaspoon vanilla extract, pinch kosher salt, 1 cup pecan halves or pieces, 1 unbaked pie crust

EASY HOMEMADE CHERRY PIE RECIPE
2 unbaked pie crust recipe, 1 cup sugar, 3 tablespoons cornstarch, 1/4 teaspoon kosher salt, 1 tablespoon butter, 1 tablespoon lemon juice, 3 cups cherries (fresh (pitted), frozen (thawed), or canned), 1 teaspoon vanilla extract (or almond extract), 1 large egg white (optional), 1 tablespoon coarse sugar (option)

COCONUT CREAM PIE RECIPE
1 pie crust recipe, 1/2 cup flaked (sweetened coconut), 1 1/2 cups coconut milk, 1 1/2 cups half-and-half, 5 egg

INI YANG SEMUANYA

In [44]:
useringreds = "corn"
usertokens = user_tokenize(useringreds)
print('User Input: ', useringreds)
print('Tokens Generated: ', usertokens, '\n')

# Vectorize user input
user_vec = vectorizer.transform([usertokens])

# Adjust weights for key ingredients
feature_names = vectorizer.get_feature_names_out()
for ingredient, weight in key_ingredients_weights.items():
    if ingredient in usertokens:
        index = feature_names.tolist().index(ingredient)
        user_vec[0, index] *= weight

# Transform user vector into topic space
topic_vec = nmf_model.transform(user_vec)

# Compute similarity and get recommendations
indices = pairwise_distances(topic_vec, doc_topic, metric='cosine').argsort().ravel()

print("Recipe Recommendations containing '{}' or its tokens:".format(useringreds))
found_recommendation = False
for index in indices:
    if any(token in df.iloc[index]['All Ingredients'].lower() for token in usertokens.split(', ')):
        print(df.iloc[index].Title.upper())
        print(df.iloc[index]["All Ingredients"])
        print()  # Print a blank line for readability
        found_recommendation = True

if not found_recommendation:
    print("No recipes found containing '{}' or its tokens.".format(useringreds))


User Input:  corn
Tokens Generated:  corn 

Recipe Recommendations containing 'corn' or its tokens:
TOFFEE PECAN PIE RECIPE
1 Pie Crust Recipe, 3 eggs, ½ cup sugar, 1 cup corn syrup, 1 teaspoon vanilla extract, 1/4 cup butter (melted), pinch kosher salt, 2 cups pecans, 1 cup toffee bits

BEST PECAN PIE RECIPE
½ cup granulated sugar, 3 large eggs, 1 cup light corn syrup or maple syrup, ½ cup butter (melted), 1 teaspoon vanilla extract, pinch kosher salt, 1 cup pecan halves or pieces, 1 unbaked pie crust

EASY HOMEMADE CHERRY PIE RECIPE
2 unbaked pie crust recipe, 1 cup sugar, 3 tablespoons cornstarch, 1/4 teaspoon kosher salt, 1 tablespoon butter, 1 tablespoon lemon juice, 3 cups cherries (fresh (pitted), frozen (thawed), or canned), 1 teaspoon vanilla extract (or almond extract), 1 large egg white (optional), 1 tablespoon coarse sugar (option)

COCONUT CREAM PIE RECIPE
1 pie crust recipe, 1/2 cup flaked (sweetened coconut), 1 1/2 cups coconut milk, 1 1/2 cups half-and-half, 5 egg yolks

In [14]:
# Simpan NMF model
with open('nmf_model.pkl', 'wb') as f:
    pickle.dump(nmf_model, f)

# Simpan vectorizer
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Simpan data doc_topic
with open('doc_topic.pkl', 'wb') as f:
    pickle.dump(doc_topic, f)

# Simpan data frame df_recipes
with open('df.pkl', 'wb') as f:
    pickle.dump(df, f)


In [15]:
#Vectorizer.pkl to json


# Ekstraksi atribut-atribut yang relevan
vectorizer_data = {
    'vocabulary_': vectorizer.vocabulary_,
    'idf_': vectorizer.idf_.tolist(),
    'tokenizer': 'commatokenizer',  # Simpan nama tokenizer sebagai referensi
}

# Simpan ke file JSON
with open('vectorizer.json', 'w') as outfile:
    json.dump(vectorizer_data, outfile, ensure_ascii=False, indent=4)